In [1]:
import pandas as pd

In [12]:
train = pd.read_csv('clean_train.csv')

In [13]:
train.columns

Index(['Stall_no', 'Market_Category', 'Loyalty_customer', 'Product_Category',
       'Grade', 'Demand', 'Discount_avail', 'charges_1', 'charges_2 (%)',
       'Minimum_price', 'Maximum_price', 'Selling_Price'],
      dtype='object')